In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from dense_vnet.DenseVNet import DenseVNet
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 12
BATCH_SIZE = 2 

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device("cuda:0")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = get_full_resolution_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = get_full_resolution_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [ ]:
from bv_refinement_network.RefinementModel import RefinementModel

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    
refine_model = RefinementModel(num_classes=1)
refine_model = nn.DataParallel(refine_model)
refine_model = convert_model(refine_model)
refine_model = refine_model.to(device, dtype)

optimizer = optim.Adam(refine_model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

deeplab = DeepLab(output_stride=16)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

checkpoint = torch.load('../deeplab_dilated_save/2019-08-10 09:28:43.844872 epoch: 1160.pth') # best one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device, dtype)

epoch = 0

In [ ]:
'''
test_dictionary = train_dataset[33]

image_1 = test_dictionary['image1_data'].view(1, 1, 256, 256, 256)
label_1 = test_dictionary['image1_label'].view(1, 3, 256, 256, 256)
bv_label = label_1.narrow(1,2,1).to(device, dtype)
if get_dimensions(bv_label) == 4:
    bv_label.unsqueeze_(0)

image_1 = image_1.to(device=device, dtype=dtype) 
label_1 = label_1.to(device=device, dtype=dtype)

deeplab.eval()

out_coarse = deeplab(image_1)
bv_coarse = out_coarse.narrow(1,2,1)

if get_dimensions(bv_coarse) == 4:
    bv_coarse.unsqueeze_(0)

seg_image_concat = torch.cat([bv_coarse, image_1], dim=1)
print(bv_label.shape)
print(bv_coarse.shape)
print(dice_loss(bv_coarse, bv_label))
print(seg_image_concat.shape)
'''

"\ntest_dictionary = train_dataset[33]\n\nimage_1 = test_dictionary['image1_data'].view(1, 1, 256, 256, 256)\nlabel_1 = test_dictionary['image1_label'].view(1, 3, 256, 256, 256)\nbv_label = label_1.narrow(1,2,1).to(device, dtype)\nif get_dimensions(bv_label) == 4:\n    bv_label.unsqueeze_(0)\n\nimage_1 = image_1.to(device=device, dtype=dtype) \nlabel_1 = label_1.to(device=device, dtype=dtype)\n\ndeeplab.eval()\n\nout_coarse = deeplab(image_1)\nbv_coarse = out_coarse.narrow(1,2,1)\n\nif get_dimensions(bv_coarse) == 4:\n    bv_coarse.unsqueeze_(0)\n\nseg_image_concat = torch.cat([bv_coarse, image_1], dim=1)\nprint(bv_label.shape)\nprint(bv_coarse.shape)\nprint(dice_loss(bv_coarse, bv_label))\nprint(seg_image_concat.shape)\n"

In [ ]:
epochs = 5000

record = open('train_bv_refine_2.txt','a+')

logger = {'train':[], 'validation_1': []}

min_val = 1

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches
    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
        #print(batch.shape)
        training_losses = []
        for minibatch in range(BATCH_SIZE):
            refine_model.train()
            deeplab.eval()
            # Set the model flag to train
            # 1. enable dropout
            # 2. batchnorm behave differently in train and test
            #print(batch['image1_data'])
            image_1 = batch['image1_data'][minibatch].to(device=device, dtype=dtype)
            image_1 = image_1.view(1,1,256,256,256)
            
            label_1 = batch['image1_label'][minibatch].to(device=device, dtype=dtype)
            label_1 = label_1.view(1,3,256,256,256)

            bv_label = label_1[:, 2, :, :, :]
            bv_label = bv_label.view(1,1,256,256,256)

            original_res = [a[minibatch].item() for a in batch['original_resolution']]
    
            # Get coarse output from deeplab model from 256 resolution input
            out_coarse = deeplab(image_1)
            out_coarse = out_coarse.view(1,3,256,256,256)

            bv_coarse = out_coarse[:, 2, :, :, :]
            bv_coarse = bv_coarse.view(1,1,256,256,256)
            bv_coarse_binarized = binarize_output(bv_coarse)
            
            bv_x, bv_y, bv_z = loadbvcenter(bv_label.view(1, 256, 256, 256))
            bbox_bv_label = bv_label.view(256, 256, 256)[bv_x-64:bv_x+64, bv_y-64:bv_y+64, bv_z-64:bv_z+64]
            bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), 128, 128, 128).to(device, dtype)
            bbox_bv_label = bbox_bv_label.view(1,1,128,128,128)
            
            pred_x, pred_y, pred_z = loadbvcenter(bv_coarse_binarized.view(1,256,256,256))
            bbox_bv = bv_coarse.view(256, 256, 256)[pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
            bbox_bv = reshape_image(bbox_bv.squeeze(), 128, 128, 128).to(device, dtype)
            bbox_bv = bbox_bv.view(1,1,128,128,128)

            #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
            bbox_image = image_1[:, :, pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
            bbox_image = reshape_image(bbox_image.squeeze(), 128, 128, 128).to(device, dtype)
            bbox_image = bbox_image.view(1, 1, 128, 128, 128)
            #print(bbox_image.shape)

            #show_image_slice(image_1)
            #show_image_slice(image_1_resize)
            #show_image_slice(label_1)
            #show_image_slice(bv_label)
            #show_image_slice(bv_label_resize)
            #show_image_slice(out_coarse)
            #show_image_slice(bv_coarse)
            #show_image_slice(bv_coarse_resize)
            #show_image_slice(bbox_bv_label)
            #show_image_slice(bbox_bv)
            #show_image_slice(bbox_image)

            bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)


            #print("bbox_concat", bbox_concat.shape)
            #seg_image_concat = torch.cat([bv_coarse, image_1], dim=1)

            del out_coarse
            del image_1
            del bv_coarse
            del label_1
            del bv_label
            del bbox_image
            del bbox_bv
            
            torch.cuda.empty_cache()

            refine_out = refine_model(bbox_concat)
            #refine_out = refine_model(seg_image_concat)
            # do the inference

            #print(refine_out.shape)
            #print(bbox_bv_label.shape)
                
            loss = dice_loss(refine_out, bbox_bv_label)
            training_losses.append(loss)
            
        # calculate loss
        avg_loss = sum(training_losses) / BATCH_SIZE
        training_losses = []
        
        print(avg_loss)
        epoch_loss += avg_loss.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        avg_loss.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        # take a gradient descent step
        torch.cuda.empty_cache()
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if e%5 == 0:
    # do validation every 5 epoches
        deeplab.eval()
        refine_model.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
            # iter over validation mini batches
                val_losses = []
                for minibatch in range(BATCH_SIZE):
                    image_1 = vbatch['image1_data'][minibatch].to(device=device, dtype=dtype)
                    image_1 = image_1.view(1,1,256,256,256)

                    label_1 = vbatch['image1_label'][minibatch].to(device=device, dtype=dtype)
                    label_1 = label_1.view(1,3,256,256,256)

                    bv_label = label_1[:, 2, :, :, :]
                    bv_label = bv_label.view(1,1,256,256,256)

                    original_res = [a[minibatch].item() for a in vbatch['original_resolution']]

                    # Get coarse output from deeplab model from 256 resolution input
                    out_coarse = deeplab(image_1)
                    out_coarse = out_coarse.view(1,3,256,256,256)

                    bv_coarse = out_coarse[:, 2, :, :, :]
                    bv_coarse = bv_coarse.view(1,1,256,256,256)
                    bv_coarse_binarized = binarize_output(bv_coarse)

                    bv_x, bv_y, bv_z = loadbvcenter(bv_label.view(1, 256, 256, 256))
                    bbox_bv_label = bv_label.view(256, 256, 256)[bv_x-64:bv_x+64, bv_y-64:bv_y+64, bv_z-64:bv_z+64]
                    bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), 128, 128, 128).to(device, dtype)
                    bbox_bv_label = bbox_bv_label.view(1,1,128,128,128)
                    
                    pred_x, pred_y, pred_z = loadbvcenter(bv_coarse_binarized.view(1,256,256,256))
                    bbox_bv = bv_coarse.view(256, 256, 256)[pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
                    #print(bbox_bv.shape)
                    bbox_bv = reshape_image(bbox_bv.squeeze(), 128, 128, 128).to(device, dtype)
                    bbox_bv = bbox_bv.view(1,1,128,128,128)

                    #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
                    bbox_image = image_1[:, :, pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
                    bbox_image = reshape_image(bbox_image.squeeze(), 128, 128, 128).to(device, dtype)
                    bbox_image = bbox_image.view(1, 1, 128, 128, 128)
                    #print(bbox_image.shape)

                    #show_image_slice(image_1)
                    #show_image_slice(image_1_resize)
                    #show_image_slice(label_1)
                    #show_image_slice(bv_label)
                    #show_image_slice(bv_label_resize)
                    #show_image_slice(out_coarse)
                    #show_image_slice(bv_coarse)
                    #show_image_slice(bv_coarse_resize)
                    #show_image_slice(bbox_bv_label)
                    #show_image_slice(bbox_bv)
                    #show_image_slice(bbox_image)

                    bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)

                    del out_coarse
                    del image_1
                    del bv_coarse
                    del label_1
                    del bv_label
                    del bbox_image
                    del bbox_bv
                    
                    torch.cuda.empty_cache()

                    refine_out = refine_model(bbox_concat)
                        
                    loss = dice_loss(refine_out, bbox_bv_label)
                    val_losses.append(loss)
                
                avg_loss = sum(val_losses) / BATCH_SIZE
                val_losses = []
                print(avg_loss)
            
                # calculate loss
                valloss_1 += avg_loss.item()
                
                torch.cuda.empty_cache()
                
            
            avg_val_loss = (valloss_1 / (v+1))
            outstr = '------- 1st valloss={0:.4f}'\
                .format(avg_val_loss) + '\n'
            
            logger['validation_1'].append(avg_val_loss)
            #scheduler.step(avg_val_loss)
            
            if avg_val_loss < min_val:
                save_1('refine_bv2_save', refine_model, optimizer, logger, e, scheduler)
            elif e % 10 == 0:
                save_1('refine_bv2_save', refine_model, optimizer, logger, e, scheduler)
            
            torch.cuda.empty_cache()
            
            print(outstr)
            record.write(outstr)
            record.flush()
    


record.close()

  0%|          | 0/4999 [00:00<?, ?it/s]

tensor(0.9731, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9746, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9782, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9645, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9818, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9829, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9717, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9745, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9818, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9678, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9733, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9752, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9765, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9870, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9860, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9742, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9707, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9691, device='cuda:0', grad_fn=<DivBack

  0%|          | 1/4999 [05:23<449:17:10, 323.62s/it]

Epoch 1 finished ! Training Loss: 0.9789

tensor(0.9825, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9780, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9901, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9713, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9786, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9938, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9684, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9812, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9716, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9804, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9769, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9730, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9737, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9763, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9788, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9908, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9971, device='cuda:0', grad_fn=<DivBackward0>)
tensor

  0%|          | 2/4999 [10:43<447:36:48, 322.48s/it]

...

Epoch 16 finished ! Training Loss: 0.9790

tensor(0.9748, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9750, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9840, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9770, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9951, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9718, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9693, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9888, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9798, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9833, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9757, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9642, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9769, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9806, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9825, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9816, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.

  0%|          | 17/4999 [1:33:41<456:46:18, 330.06s/it]

Epoch 17 finished ! Training Loss: 0.9790

tensor(0.9756, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9836, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9742, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9798, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9749, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9806, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9727, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9665, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9830, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9817, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9776, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9861, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9801, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9760, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9767, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9725, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.

  0%|          | 18/4999 [1:38:59<451:50:34, 326.57s/it]

Epoch 18 finished ! Training Loss: 0.9782

tensor(0.9707, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9744, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9828, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9779, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9756, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9896, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9819, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9879, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9898, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9752, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9752, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9704, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9766, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9922, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9750, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9780, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.

In [ ]:
print(image_1_resize.shape)

In [ ]:
show_image_slice(image_1)
show_image_slice(label_1)
show_image_slice(bbox_bv_label)
show_image_slice(out_coarse)
show_image_slice(bbox_bv)
show_image_slice(bbox_image)

In [ ]:
b = loadbvcenter(image_1.squeeze(0).cpu().detach().numpy())
x1 = int(b[0])
y1 = int(b[1])
z1 = int(b[2])
show_image_slice(bv_label[x1-64:x1+64, y1-64:y1+64, z1-64:z1+64])
c = find_bv_centroid(bv_label)
x2 = int(c[0])
y2 = int(c[1])
z2 = int(c[2])
show_image_slice(bv_label[x2-64:x2+64, y2-64:y2+64, z2-64:z2+64])

In [ ]:
print(bv_coarse.shape)
print("bbox_bv_label", bbox_bv_label.shape)
print("bbox_bv", bbox_bv.shape)
print("bbox_image", bbox_image.shape)

In [ ]:
deeplab.eval()

with torch.no_grad():
    
    bgloss = 0
    bdloss = 0
    bvloss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
        # move data to device, convert dtype to desirable dtype
        image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
        label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)

        output = deeplab(image_1)
        # do the inference
        output_numpy = output.cpu().numpy()
        
        
        #out_1 = torch.round(output)
        out_1 = torch.from_numpy((output_numpy == output_numpy.max(axis=1)[:, None]).astype(int)).to(device=device, dtype=dtype)
        loss_1 = dice_loss_3(out_1, label_1)

        bg, bd, bv = dice_loss_3_debug(out_1, label_1)
        # calculate loss
        print(bg.item(), bd.item(), bv.item(), loss_1.item())
        bgloss += bg.item()
        bdloss += bd.item()
        bvloss += bv.item()
        
        if bv.item() >= 0.2 or bd.item() >= 0.1:
            show_image_slice(image_1)
            show_image_slice(label_1)
            show_image_slice(output)

    outstr = '------- background loss = {0:.4f}, body loss = {1:.4f}, bv loss = {2:.4f}'\
        .format(bgloss/(v+1), bdloss/(v+1), bvloss/(v+1)) + '\n'
    print(outstr)